# Analysis script for results in the paper

In [1]:
# import libraries

import scipy
from scipy.stats import sem as sem
import sys
import glob
import pandas as pd

if "/home/anna/Documents/cinnabar" not in sys.path:
    sys.path.insert(1, "/home/anna/Documents/cinnabar")
import cinnabar

print("adding code to the pythonpath...")
code = "/home/anna/Documents/code/python"
if code not in sys.path:
    sys.path.insert(1, code)
import pipeline

print(cinnabar.__file__)

from pipeline import *
from pipeline.utils import validate
from pipeline.analysis import *

adding code to the pythonpath...
[Errno 2] No such file or directory: '/home/anna/anaconda3/envs/biosimspace-dev/bin/pmemd.cuda'
could not determine AMBER version.

Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.



Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


/home/anna/Documents/cinnabar/cinnabar/__init__.py


load in the data for the protein system


In [2]:
bench_folder = f"/home/anna/Documents/benchmark"
protein = "tyk2"
main_dir = f"{bench_folder}/extracted/{protein}"
# main_dir = f"/backup/manual_reruns/{protein}"

ana_obj_dict = {}

# for the different networks
for net in ["lomap", "rbfenn", "combined"]:
    # choose location for the files
    net_file = f"{main_dir}/execution_model/network_{net}.dat"
    ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"  # can also cycle through different analysis protocols
    exp_file = f"{bench_folder}/inputs/experimental/{protein}.yml"

    results_folder = f"{main_dir}/outputs_extracted/results"
    output_folder = validate.folder_path(
        f"{main_dir}/analysis_paper_test/{net}", create=True
    )

    ana_obj = analysis_network(
        results_folder,
        exp_file=exp_file,
        net_file=net_file,
        output_folder=output_folder,
        analysis_prot=ana_file,
    )

    # Add ligands folder for drawing

    ana_obj.add_ligands_folder(f"{bench_folder}/inputs/{protein}/ligands")

    ana_obj_dict[net] = ana_obj

In [3]:
# analyse each object

for ana_obj in ana_obj_dict.values():
    # compute results
    ana_obj.compute_results()

/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home

SVD did not converge
could not create cinnabar network for AMBER
SVD did not converge
could not create cinnabar network for GROMACS
SVD did not converge
could not create cinnabar network for AMBER
SVD did not converge
could not create cinnabar network for GROMACS
SVD did not converge
could not create cinnabar network for AMBER
SVD did not converge
could not create cinnabar network for GROMACS


### reproducibility between perturbations

this is for the combined networks

check:
vs experimental
vs each other
outliers for each engine
most different perturbations

In [ ]:
ana_obj = ana_obj_dict["combined"]

# compared to each other
mad_df, mad_df_err = ana_obj.calc_mad_engines(pert_val="pert")
print(mad_df)
print(mad_df_err)

# # compared to experimental
mae_df, mae_df_err = ana_obj.calc_mae_engines(pert_val="pert")
print(mae_df)
print(mae_df_err)

# also saved in output folder

checking results, convergence, spread of data between engines

In [ ]:
for ana_obj in ana_obj_dict.values():
    # can compute convergence for all
    ana_obj.compute_convergence(main_dir=main_dir)
    ana_obj.plot_convergence()

In [ ]:
# histograms

for ana_obj in ana_obj_dict.values():
    ana_obj.plot_histogram_repeats()

    ana_obj.plot_histogram_legs()

    ana_obj.plot_histogram_sem()

outliers for each engine

In [ ]:
ana_obj.plot_outliers(no_outliers=3)

In [ ]:
for eng in ana_obj.engines:
    # get outliers above a certain threshold
    perts = ana_obj.get_outliers(threshold=3, name=eng)
    print(f"{eng} : {perts}")

    # draw the perturbations
    ana_obj.draw_perturbations(perts)

perturbations that are the most different between the engines

In [ ]:
all_engines_list_dict = {}

for pert in ana_obj.perturbations:
    all_engines_list_dict[pert] = []

    for eng in ana_obj.engines:
        all_engines_list_dict[pert].append(ana_obj.calc_pert_dict[eng][pert][0])

In [ ]:
def find_top_largest_difference_pert(dictionary_of_lists, num_ranges):
    ranges = {}

    for key, sublist in dictionary_of_lists.items():
        range = max(sublist) - min(sublist)
        ranges[key] = range

    sorted_ranges = sorted(ranges.items(), key=lambda x: x[1], reverse=True)
    top_ranges = sorted_ranges[:num_ranges]

    return top_ranges

In [ ]:
top_ranges = find_top_largest_difference_pert(all_engines_list_dict, 3)
print(top_ranges)
perts = [a[0] for a in top_ranges]
ana_obj.draw_perturbations(perts)

the different analysis methods (MBAR/TI, % of run used, stats ineff, autoeq)

In [ ]:
# code

### reproducibility between ligs - per ligand results

this is for the individual networks

In [ ]:
# plot for each network and calc mad / mae


for net in ana_obj_dict:
    print(net)
    ana_obj = ana_obj_dict[net]

    stats = ana_obj.calc_stats()

    title = ""
    title += f"{net}"

    # plotting with r2, spearmans rank
    for eng in ana_obj.engines:
        mue_exp = r2val = stats["val"]["experimental"][eng]["MUE"]
        r2val = stats["val"]["experimental"][eng]["R2"]
        rmseval = stats["val"]["experimental"][eng]["RMSE"]
        spearman = stats["val"]["experimental"][eng]["rho"]

        # titles
        title += f"\n MAE : {mue_exp[0]:.2f} +/- {mue_exp[1]:.2f} kcal/mol"
        title += f"\n R2 : {r2val[0]:.2f} +/- {r2val[1]:.2f} kcal/mol"
        title += f"\n RMSE : {rmseval[0]:.2f} +/- {rmseval[1]:.2f} kcal/mol"
        title += f"\n rho : {spearman[0]:.2f} +/- {spearman[1]:.2f} kcal/mol"

        kwargs = {"title": title}
        ana_obj.plot_scatter_lig(engine=eng, **kwargs)

    # compared to each other
    mad_df, mad_df_err = ana_obj.calc_mad_engines(pert_val="val")
    print(mad_df)
    print(mad_df_err)

    # # compared to experimental
    mae_df, mae_df_err = ana_obj.calc_mae_engines(pert_val="val")
    print(mae_df)
    print(mae_df_err)

    # also saved in output folder

### different network analysis methods

all so far with cinnabar.

In [4]:
# compared to fwf

for net in ana_obj_dict:
    print(net)
    ana_obj = ana_obj_dict[net]

    # first need to add the fwf path
    ana_obj._add_fwf_path(
        "/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis"
    )

    title = ""
    title += f"{net}"

    for eng in ana_obj.engines:
        # get the network analysis
        fwf_dict = ana_obj._get_ana_fwf(engine=eng)
        # for key in fwf_dict:
        #     print(f"{key} : {fwf_dict[key][0]}, {fwf_dict[key][1]}")

        # get fwf stats
        fwf_stats = ana_obj._get_stats_fwf(engine=eng)

        # compared to each other
        mad_df, mad_df_err = ana_obj._get_mad_fwf(ana_obj.engines, ana_obj.engines)
        print(mad_df)
        print(mad_df_err)

lomap


Added additional data to 38 edges; added 0 new edges.
Added additional data to 37 edges; added 0 new edges.
R confidence is:   0.72 < 0.74 < 0.76
MUE confidence is: 0.68 < 0.70 < 0.73
Tau confidence is: 0.49 < 0.50 < 0.53
Added additional data to 38 edges; added 0 new edges.
Added additional data to 37 edges; added 0 new edges.
Added additional data to 38 edges; added 0 new edges.
Added additional data to 37 edges; added 0 new edges.
Added additional data to 38 edges; added 0 new edges.
Added additional data to 37 edges; added 0 new edges.
Added additional data to 43 edges; added 1 new edges.
Added additional data to 43 edges; added 1 new edges.
Added additional data to 38 edges; added 0 new edges.
Added additional data to 37 edges; added 0 new edges.
Added additional data to 31 edges; added 10 new edges.
Added additional data to 31 edges; added 4 new edges.
Added additional data to 43 edges; added 1 new edges.
Added additional data to 43 edges; added 1 new edges.
Added additional data

In [ ]:
# compared to mbarnet

### consensus

consensus scoring of the engines

In [ ]:
# get average of averages

for net in ana_obj_dict:
    ana_obj = ana_obj_dict[net]

    consensus_pert_dict = {}

    for pert in ana_obj.perturbations:
        consensus_pert_dict[pert] = []

        for eng in ana_obj.engines:
            consensus_pert_dict[pert].append(ana_obj.calc_pert_dict[eng][pert][0])

        consensus_pert_dict[pert] = (
            np.mean(consensus_pert_dict[pert]),
            sem(consensus_pert_dict[pert]),
        )

        df = pd.DataFrame.from_dict(
            consensus_pert_dict, orient="index", columns=["freenrg", "error"]
        )
        df.index.name = "perturbations"
        df = df.reset_index()
        df[["lig_0", "lig_1"]] = df["perturbations"].str.split("~", expand=True)
        df["engine"] = "consensus"
        df["analysis"] = ana_obj.file_ext
        df["method"] = "None"
        df = df.drop(labels="perturbations", axis=1)
        df = df[["lig_0", "lig_1", "freenrg", "error", "engine", "analysis", "method"]]

        df.to_csv(
            f"{ana_obj.output_folder}/consensus_perturbations.csv", sep=",", index=False
        )

    ana_obj.compute_other_results(
        f"{ana_obj.output_folder}/consensus_perturbations.csv", name="consensus"
    )

    print(net)

In [ ]:
# consensus scoring, is it more robust

for net in ana_obj_dict:
    print(net)

    ana_obj = ana_obj_dict[net]

    for pv in ["pert", "val"]:
        print(pv)
        mae_df, mae_df_err = ana_obj.calc_mae_engines(pv, engines="consensus")
        print(mae_df)
        print(mae_df_err)

### comparing different networks

for the lomap and the rbfenn 

In [6]:
error_val_dict.values()

dict_values([0.2138, 0.0815, nan, 0.1556, 0.0472, 0.0548, 0.0756, 0.0817, 0.1691, nan, 0.0087, 0.095, 0.0545, 0.0419, 0.0239, 0.0157, 0.0716, nan, 0.0294, 0.0628, 0.1171, 0.0298, 0.0619, nan])

In [10]:
# check if the error for the different networks is lower

error_dict = {}
diff_to_exp_dict = {}

for net in ana_obj_dict:
    print(net)

    ana_obj = ana_obj_dict[net]

    error_dict[net] = {}
    diff_to_exp_dict[net] = {}

    for eng in ana_obj.engines:
        error_val_dict = {}
        diff_to_exp_val_dict = {}

        for key in ana_obj.calc_pert_dict[eng]:
            error_val_dict[key] = ana_obj.calc_pert_dict[eng][key][1]
            diff_to_exp_val_dict[key] = abs(
                ana_obj.calc_pert_dict[eng][key][0] - ana_obj.exper_pert_dict[key][0]
            )

        # test for normal distribution in the errors
        # if less than 0.05, not normal
        print(
            "error",
            scipy.stats.shapiro(
                list(filter(lambda item: item is not None, error_val_dict.values()))
            ),
        )
        print(
            "diff to exp",
            scipy.stats.shapiro(
                list(
                    filter(lambda item: item is not None, diff_to_exp_val_dict.values())
                )
            ),
        )

        avg_error = np.mean(
            list(filter(lambda item: item is not None, error_val_dict.values()))
        )
        avg_diff_exp = np.mean(
            list(filter(lambda item: item is not None, diff_to_exp_val_dict.values()))
        )

        print(f"{net}, {eng}, avg error is : {avg_error}")
        print(f"{net}, {eng}, avg diff to exp is : {avg_diff_exp}")

        error_dict[net][eng] = error_val_dict
        diff_to_exp_dict[net][eng] = diff_to_exp_val_dict

lomap
error ShapiroResult(statistic=nan, pvalue=1.0)
diff to exp ShapiroResult(statistic=nan, pvalue=1.0)
lomap, AMBER, avg error is : nan
lomap, AMBER, avg diff to exp is : nan
error ShapiroResult(statistic=0.7854276895523071, pvalue=0.00017182380543090403)
diff to exp ShapiroResult(statistic=0.9248272776603699, pvalue=0.07474003732204437)
lomap, SOMD, avg error is : 0.2332
lomap, SOMD, avg diff to exp is : 0.9889769206732604
error ShapiroResult(statistic=nan, pvalue=1.0)
diff to exp ShapiroResult(statistic=nan, pvalue=1.0)
lomap, GROMACS, avg error is : nan
lomap, GROMACS, avg diff to exp is : nan
rbfenn
error ShapiroResult(statistic=nan, pvalue=1.0)
diff to exp ShapiroResult(statistic=nan, pvalue=1.0)
rbfenn, AMBER, avg error is : nan
rbfenn, AMBER, avg diff to exp is : nan
error ShapiroResult(statistic=0.8208369612693787, pvalue=0.00016104488167911768)
diff to exp ShapiroResult(statistic=0.8085482716560364, pvalue=9.422206494491547e-05)
rbfenn, SOMD, avg error is : 0.21578
rbfenn, 

In [12]:
# mann whitney u, as not normally distributed

for eng in ana_obj.engines:
    group1 = list(
        filter(lambda item: item is not None, error_dict["lomap"][eng].values())
    )
    group2 = list(
        filter(lambda item: item is not None, error_dict["rbfenn"][eng].values())
    )

    ustats, pvalue = scipy.stats.mannwhitneyu(group1, group2)

    print(f"{eng}: {ustats, pvalue}")

# if below 0.05 (if confidence interval) there is significant difference (reject null hypothesis)

AMBER: (nan, nan)
SOMD: (366.0, 0.9237104217365618)
GROMACS: (nan, nan)


In [13]:
# check difference to experimental sig diff between the two

for eng in ana_obj.engines:
    group1 = list(
        filter(lambda item: item is not None, diff_to_exp_dict["lomap"][eng].values())
    )
    group2 = list(
        filter(lambda item: item is not None, diff_to_exp_dict["rbfenn"][eng].values())
    )

    ustats, pvalue = scipy.stats.mannwhitneyu(group1, group2)

    print(f"{eng}: {ustats, pvalue}")

AMBER: (nan, nan)
SOMD: (293.0, 0.2469305203090758)
GROMACS: (nan, nan)


### directionality

data from featurising the perturbations

In [24]:
ana_obj = ana_obj_dict["combined"]

for eng in ana_obj.engines:
    error_dict = {
        key: ana_obj.calc_pert_dict[eng][key][1] for key in ana_obj.calc_pert_dict[eng]
    }
    df = pd.read_csv(f"{main_dir}/execution_model/grow_shrink_featurise.dat")
    df[f"error_{eng}"] = df["pert"].map(error_dict)
    df = df.dropna()

df.loc["grow/shrink" == "grow"]

KeyError: 'False: boolean label can not be used without a boolean index'